# Laboratorio #5 – Análisis de tráfico, implementación de un IDS



### 0. Analisis de paquetes scapy

#### Importacion de librerias

In [1]:
!pip install scapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444345 sha256=a0cdb26d22ad0ed19cab565878d6737382d3d10dcd00313adcf0fe503beb44b1
  Stored in directory: /root/.cache/pip/wheels/dd/1b/47/d46b1a87e339be501612cf4cd1bf57742e534f9c9aac7b00d6
Successfully built scapy


In [2]:
import pandas as pd
import numpy as np 
import binascii
import seaborn as sns

from scapy.all import *

In [3]:
sns.set(color_codes=True)
%matplotlib inline

### Captura de paquetes

In [4]:
num = 100
pcap = sniff(count=num)

print(type(pcap))
print(len(pcap))
print(pcap)
pcap[0]


<class 'scapy.plist.PacketList'>
100
<Sniffed: TCP:100 UDP:0 ICMP:0 Other:0>


<Ether  dst=02:42:ac:1c:00:0c src=02:42:81:d6:dd:bc type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=522 id=39575 flags=DF frag=0 ttl=64 proto=6 chksum=0x4611 src=172.28.0.1 dst=172.28.0.12 |<TCP  sport=34096 dport=8080 seq=3178069003 ack=3959747405 dataofs=8 reserved=0 flags=PA window=501 chksum=0x5a42 urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (937786900, 3639753796))] |<Raw  load='GET /socket.io/?EIO=3&sid=R_PmmLqmwiabq0EaAAAA&t=OU89SGQ&transport=polling HTTP/1.1\r\nHost: colab.research.google.com\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/111.0,gzip(gfe)\r\nAccept: */*\r\nAccept-Encoding: gzip, deflate, br,gzip(gfe)\r\nAccept-Language: es-ES,es;q=0.8,en-US;q=0.5,en;q=0.3\r\nReferer: https://colab.research.google.com/\r\nSec-Fetch-Dest: empty\r\nSec-Fetch-Mode: cors\r\nSec-Fetch-Site: same-origin\r\nTe: trailers\r\n\r\n' |>>>>

### informacion de paquetes capturados

In [6]:
#informacion
e_frame = pcap[0]
ip_paquete = e_frame.payload
segmento = ip_paquete.payload
data = segmento.payload

print("Ethernet frame:")
print(e_frame.show())
print()

print("Paquete:")
print(ip_paquete.show())
print()

print("Segmento:")
print(segmento.show())
print()

print("Data:")
print(data.show())
print()

Ethernet frame:
###[ Ethernet ]### 
  dst       = 02:42:ac:1c:00:0c
  src       = 02:42:81:d6:dd:bc
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 522
     id        = 39575
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = 6
     chksum    = 0x4611
     src       = 172.28.0.1
     dst       = 172.28.0.12
     \options   \
###[ TCP ]### 
        sport     = 34096
        dport     = 8080
        seq       = 3178069003
        ack       = 3959747405
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 501
        chksum    = 0x5a42
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (937786900, 3639753796))]
###[ Raw ]### 
           load      = 'GET /socket.io/?EIO=3&sid=R_PmmLqmwiabq0EaAAAA&t=OU89SGQ&transport=polling HTTP/1.1\r\nHost: colab.research.google.com\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gec

### Tipos de datos

In [8]:
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
from scapy.layers.inet import TCP, UDP

#datos
print(type(e_frame))
print(type(ip_paquete))
print(type(segmento))
print()

ethernet_type = type(e_frame)
ip_type = type(ip_paquete)
tcp_type = type(segmento)

#por segmentos
print("Ethernet",pcap[ethernet_type])
print()
print("IP", pcap[ip_type])
print()
print("TCP", pcap[tcp_type])
print()
print("UDP", pcap[UDP])

<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.inet.IP'>
<class 'scapy.layers.inet.TCP'>

Ethernet <Ether from Sniffed: TCP:100 UDP:0 ICMP:0 Other:0>

IP <IP from Sniffed: TCP:100 UDP:0 ICMP:0 Other:0>

TCP <TCP from Sniffed: TCP:100 UDP:0 ICMP:0 Other:0>

UDP <UDP from Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>


### Preparacion data set

In [11]:
#IP/TCP/UDP como columnas
ip_column = [field.name for field in IP().fields_desc]
tcp_column = [field.name for field in TCP().fields_desc]
udp_column = [field.name for field in UDP().fields_desc]

#Conquetanar los campos
dataframe_column = ip_column + ['time'] + tcp_column + ['payload','payload_raw','payload_hex']

# Creacion de data set
df = pd.DataFrame(columns=dataframe_column)
for packet in pcap[IP]:
    field_val = []
    for field in ip_column:
        if field == 'options':
            field_val.append(len(packet[IP].fields[field]))
        else:
            field_val.append(packet[IP].fields[field])
    
    field_val.append(packet.time)
    
    layer_type = type(packet[IP].payload)
    for field in tcp_column:
        try:
            if field == 'options':
                field_val.append(len(packet[layer_type].fields[field]))
            else:
                field_val.append(packet[layer_type].fields[field])
        except:
            field_val.append(None)
    
    #payload
    field_val.append(len(packet[layer_type].payload))
    field_val.append(packet[layer_type].payload.original)
    field_val.append(binascii.hexlify(packet[layer_type].payload.original))

    df_append = pd.DataFrame([field_val], columns=dataframe_column)
    df = pd.concat([df, df_append], axis=0)

df = df.reset_index()
df = df.drop(columns="index")

#head data set
print(df.head(3))

  version ihl tos   len     id flags frag ttl proto chksum  ... dataofs  \
0       4   5   0   522  39575    DF    0  64     6  17937  ...       8   
1       4   5   0    52  65360    DF    0  64     6  58157  ...       8   
2       4   5   0  1501  65361    DF    0  64     6  56707  ...       8   

  reserved flags  window chksum urgptr options payload  \
0        0    PA     501  23106      0       3     470   
1        0     A     501  22636      0       3       0   
2        0    PA     501  24085      0       3    1449   

                                         payload_raw  \
0  b'GET /socket.io/?EIO=3&sid=R_PmmLqmwiabq0EaAA...   
1                                                b''   
2  b'HTTP/1.1 200 OK\r\nContent-Type: text/plain;...   

                                         payload_hex  
0  b'474554202f736f636b65742e696f2f3f45494f3d3326...  
1                                                b''  
2  b'485454502f312e3120323030204f4b0d0a436f6e7465...  

[3 rows x 28 columns

In [15]:
df[['src','dst','sport','dport']]

,src,dst,sport,dport
0,172.28.0.1,172.28.0.12,34096,8080
1,172.28.0.12,172.28.0.1,8080,34096
2,172.28.0.12,172.28.0.1,8080,34096
3,172.28.0.1,172.28.0.12,34096,8080
4,172.28.0.1,172.28.0.12,34096,8080
...,...,...,...,...
95,172.28.0.12,172.28.0.1,8080,45808
96,172.28.0.1,172.28.0.12,45808,8080
97,172.28.0.12,172.28.0.1,8080,45808
98,172.28.0.12,172.28.0.1,8080,45808


### Estadisticas 

In [16]:
#IP de origen mas frecuente?
frequent_address = df['src'].describe()['top']


# A que IPs les habla la IP mas frecuente
print(df[df['src'] == frequent_address]['dst'].unique(),"\n\n")

['172.28.0.12'] 




In [17]:
print(df)

   version ihl tos   len     id flags frag ttl proto chksum  ... dataofs  \
0        4   5   0   522  39575    DF    0  64     6  17937  ...       8   
1        4   5   0    52  65360    DF    0  64     6  58157  ...       8   
2        4   5   0  1501  65361    DF    0  64     6  56707  ...       8   
3        4   5   0    52  39576    DF    0  64     6  18406  ...       8   
4        4   5   0   626  39577    DF    0  64     6  17831  ...       8   
..     ...  ..  ..   ...    ...   ...  ...  ..   ...    ...  ...     ...   
95       4   5   0    52  47307    DF    0  64     6  10675  ...       8   
96       4   5   0   364  14972    DF    0  64     6  42698  ...       8   
97       4   5   0    52  47308    DF    0  64     6  10674  ...       8   
98       4   5   0   337  47309    DF    0  64     6  10388  ...       8   
99       4   5   0   364  65378    DF    0  64     6  57827  ...       8   

   reserved flags  window chksum urgptr options payload  \
0         0    PA     501  2

In [18]:
from google.colab import files

df.to_csv('paquetes.csv', encoding = 'utf-8-sig') 
files.download('paquetes.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>